##### Copyright 2020 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Qubits

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/qubits"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/qubits.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/qubits.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/qubits.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

installing cirq...
     |████████████████████████████████| 1.8MB 6.2MB/s 
     |████████████████████████████████| 1.3MB 31.3MB/s 
installed cirq.


A qubit is the basic unit of quantum information, a quantum bit: a two level system that can exist in superposition of those two possible states. Cirq also supports higher dimensional systems, so called [qudits](qudits.ipynb) that we won't cover here.

In Cirq, a `Qubit` is nothing else than an abstract object that has an identifier, a `cirq.Qid` and some other potential metadata to represent device specific properties that can be used to validate a circuit.
In contrast to real qubits, the Cirq qubit does not have any state. The reason for this is that the actual state of the qubit or qubits is maintained in the quantum processor, or, in case of simulation, in the simulated state vector.

In [3]:
import cirq

qubit = cirq.NamedQubit("myqubit")

# creates an equal superposition of |0> and |1> when simulated
circuit = cirq.Circuit(cirq.H(qubit))

# see the "myqubit" identifier at the left of the circuit
print(circuit)

# run simulation
result = cirq.Simulator().simulate(circuit)

print("result:")
print(result)

myqubit: ───H───
result:
measurements: (no measurements)
output vector: 0.707|0⟩ + 0.707|1⟩


In [4]:
## Programming Quantum Computers
##   by Eric Johnston, Nic Harrigan and Mercedes Gimeno-Segovia
##   O'Reilly Media
##
## More samples like this can be found at http://oreilly-qc.github.io
import cirq

## This sample generates a single random bit.

## Example 2-1: Random bit
# Set up the program

def main():
    qc = QPU()
    qc.reset(1)
    qc.had()  # put it into a superposition of 0 and 1
    qc.read() # read the result as a digital bit

    qc.draw() # draw the circuit
    result = qc.run() # run the circuit
    print(result)


######################################################################
## The below class is a light interface, to convert the
## book's syntax into the syntax used by Cirq.
class QPU:
    def __init__(self):
        self.circuit = cirq.Circuit()
        self.simulator = cirq.Simulator()
        self.qubits = None

    def reset(self, num_qubits):
        self.qubits = [cirq.GridQubit(i, 0) for i in range(num_qubits)]

    def mask_to_list(self, mask):
        return [q for i,q in enumerate(self.qubits) if (1 << i) & mask]

    def had(self, target_mask=~0):
        target = self.mask_to_list(target_mask)
        self.circuit.append(cirq.H.on_each(*target))

    def read(self, target_mask=~0, key=None):
        if key is None:
            key = 'result'
        target = self.mask_to_list(target_mask)
        self.circuit.append(cirq.measure(*target, key=key))

    def draw(self):
        print('Circuit:\n{}'.format(self.circuit))

    def run(self):
        return self.simulator.simulate(self.circuit)


if __name__ == '__main__':
    main()

Circuit:
(0, 0): ───H───M('result')───
measurements: result=0
output vector: |0⟩


### Qubit types

There are 3 main qubit types in Cirq:

- `cirq.NamedQubit` - an abstract qubit that only has a name, nothing else. Use this when you don't need anything else and you don't need to create too many qubits in bulk.
- `cirq.LineQubit` - a qubit that is identified by an integer index in a line. Some devices have lines of qubits, `LineQubit` can be useful to represent that. Also `cirq.LineQubit.range(3)` is a very easy way to create 3 qubits.  
- `cirq.GridQubit` - a qubit that is placed on a grid and is identified by the 2D coordinates. Most of Google's chips are represented using `GridQubit`s. 

Some providers provide their own qubit types. For example Pasqal defines a `TwoDQubit` and a `ThreeDQubit` to represent the specific topology of neutral atoms when validating circuits.
